# Azure AI Agent service - Function calling

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects
#%pip install yfinance

In [2]:
import glob
import json
import ipyplot
import matplotlib.pyplot as plt
import os
import requests
import sys
import time
import yfinance as yf

#from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient 
from azure.identity import DefaultAzureCredential
#from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from azure.ai.agents.models import (
    FunctionTool,
    ListSortOrder,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
)
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Any, Callable, Set, Dict, List, Optional
from PIL import Image
from pathlib import Path

from user_functions import user_functions, azuremaps_weather

In [3]:
load_dotenv("azure.env")

True

## 1. Agent definition - Weather forecasts

## Connect to AI Foundry Project

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


### Testing the Azure Maps Weather function

We will use the Azure Maps service to provide weather information.

In [5]:
weather = azuremaps_weather("Toronto")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-06-09T16:22:00-04:00',
    'phrase': 'Mostly cloudy',
    'iconCode': 6,
    'hasPrecipitation': False,
    'isDayTime': True,
    'temperature': {'value': 17.2, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 18.3, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 15.6, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 87,
    'dewPoint': {'value': 15.0, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 180.0, 'localizedDescription': 'S'},
     'speed': {'value': 14.8, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 18.3, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 3,
    'uvIndexPhrase': 'Moderate',
    'visibility': {'value': 14.5, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': '',
    'cloudCover': 76,
    'ceiling': {'value': 4267.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1005.6, 'unit': 'mb', 'unitType': 14},
    '

### Agent

In [6]:
model = "gpt-4o-mini"
name = "gpt-4o-mini-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

functions = FunctionTool(user_functions)

# Create an agent and run user's request with function calls
agent = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name=name,
    instructions=instructions,
    tools=functions.definitions,
)
print(f"Created agent, ID: {agent.id}")



Created agent, ID: asst_FCLBFiOWAlhV0xeDqTfASuWA


### Tests

In [7]:
prompt = "Hello, generate a full report for the weather in Toronto today"

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_jlpXaap2fSs4YeZN3AmPNck7
Created message ID = msg_aBD5OV1APxPvHVBLqtEVsBos


In [8]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_KBjIH6iR8HFvwamkGSki0KK2


In [9]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_pJ99Xih6kAN5bZPzoKjcno3a', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Toronto"}'}}
Tool outputs: [{'tool_call_id': 'call_pJ99Xih6kAN5bZPzoKjcno3a', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T16:22:00-04:00", "phrase": "Mostly cloudy", "iconCode": 6, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 17.2, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 18.3, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 15.6, "unit": "C", "unitType": 17}, "relativeHumidity": 87, "dewPoint": {"value": 15.0, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 180.0, "localizedDescription": "S"}, "speed": {"value": 14.8, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 18.3, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 14.5, "unit": "km", "unitType": 6}, "obstructionsTo

In [10]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 20:29:39+0000
End     : 2025-06-09 20:29:43+0000
Elapsed : 0:00:04  (4.00 seconds)


{'prompt_tokens': 1427, 'completion_tokens': 325, 'total_tokens': 1752, 'prompt_token_details': {'cached_tokens': 0}}

In [11]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Toronto today
MessageRole.AGENT: Here is the full weather report for Toronto today:

- Condition: Mostly cloudy
- Temperature: 17.2°C (feels like 18.3°C; in the shade, 15.6°C)
- Humidity: 87%
- Dew Point: 15.0°C
- Wind: South at 14.8 km/h, gusts up to 18.3 km/h
- UV Index: 3 (Moderate)
- Visibility: 14.5 km
- Cloud Cover: 76%
- Ceiling: 4267 meters
- Barometric Pressure: 1005.6 mb (steady)
- Precipitation:
  - Past hour: 0.0 mm
  - Past 3 hours: 0.8 mm
  - Past 6 hours: 3.0 mm
  - Past 24 hours: 4.2 mm
- Temperature summary:
  - Past 6 hours: Min 15.0°C, Max 19.0°C
  - Past 12 hours: Min 13.9°C, Max 19.0°C
  - Past 24 hours: Min 13.9°C, Max 19.5°C

Today in Toronto is mostly cloudy, with cool conditions and high humidity. There has been light rainfall recently, but currently there is no precipitation. The wind is moderate from the south, and conditions should be comfortable for most outdoor activities, but UV protection

### Other examples

In [12]:
prompt = "Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_Tm7mFvA7BrtMrJpIf4ig8TJp
Created message ID = msg_C93Mu2Taa9f0pKmvnppnImeM


In [13]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_IDnExIPHax2LGUnFPx4yH364


In [14]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_wkUTRHDs4PVq2nzQfcoxU3tv', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Mexico City"}'}}
Tool outputs: [{'tool_call_id': 'call_wkUTRHDs4PVq2nzQfcoxU3tv', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T14:20:00-06:00", "phrase": "Partly sunny", "iconCode": 3, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 28.9, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 34.0, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 27.9, "unit": "C", "unitType": 17}, "relativeHumidity": 21, "dewPoint": {"value": 5.0, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 0.0, "localizedDescription": "N"}, "speed": {"value": 5.6, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 5.6, "unit": "km/h", "unitType": 7}}, "uvIndex": 8, "uvIndexPhrase": "Very High", "visibility": {"value": 16.1, "unit": "km", "unitType": 6}, "obstructionsToV

In [15]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 20:29:49+0000
End     : 2025-06-09 20:29:54+0000
Elapsed : 0:00:05  (5.00 seconds)


{'prompt_tokens': 1453, 'completion_tokens': 595, 'total_tokens': 2048, 'prompt_token_details': {'cached_tokens': 0}}

In [16]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French.
MessageRole.AGENT: English:
Today in Mexico City, the weather is partly sunny. The current temperature is 28.9°C, with a "feels like" (real feel) temperature of 34.0°C. In the shade, it feels like 27.9°C. The relative humidity is low at 21%, and the dew point is 5.0°C. Winds are coming from the north at 5.6 km/h, with no significant gusts. There is no precipitation today, and visibility is excellent at 16.1 km. The UV index is very high at 8—take protection from the sun. The sky is 35% covered with clouds, and the atmospheric pressure is 1023.7 mb, currently falling. Over the last 24 hours, the temperature ranged between 14.9°C (minimum) and 28.9°C (maximum).

Español:
Hoy en la Ciudad de México, el clima está parcialmente soleado. La temperatura actual es de 28.9°C, con una sensación térmica de 34.0°C. A la sombra, la sensación es de 27.9°C. La humeda

In [17]:
prompt = "Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_Q4E6PZ6cGZtxF4w9E6U1YwDu
Created message ID = msg_tP0PyYat52B4n5VCbekM3huc


In [18]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_4RaKyxFRE8aZM2uRgqv5K2gQ


In [19]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_yyPLlf2Lz8AUka299UCi85G1', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Oakville, Ontario"}'}}
Tool outputs: [{'tool_call_id': 'call_yyPLlf2Lz8AUka299UCi85G1', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T16:22:00-04:00", "phrase": "Mostly cloudy", "iconCode": 6, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 19.4, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 21.1, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 18.3, "unit": "C", "unitType": 17}, "relativeHumidity": 89, "dewPoint": {"value": 17.6, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 203.0, "localizedDescription": "SSW"}, "speed": {"value": 9.0, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 18.8, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 20.9, "unit": "km", "unitType": 6}, "obs

In [20]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 20:30:00+0000
End     : 2025-06-09 20:30:01+0000
Elapsed : 0:00:01  (1.00 seconds)


{'prompt_tokens': 1453, 'completion_tokens': 69, 'total_tokens': 1522, 'prompt_token_details': {'cached_tokens': 0}}

In [21]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data.
MessageRole.AGENT: The UV information for Oakville, Ontario today is as follows:

- Date of weather data: 2025-06-09
- UV Index: 3 (Moderate)

If you need additional weather details, let me know!


In [22]:
prompt = "Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_yrvl5cAxHWREWtnkT0M8v64f
Created message ID = msg_mTU1Pq47wfqs0dKPn7A9qoMO


In [23]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_kC1kGT8zagkPXqFdchuzRZTp


In [24]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_pFlxXQanw2j67eyfN1W3VJ1j', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Rome"}'}}
Tool outputs: [{'tool_call_id': 'call_pFlxXQanw2j67eyfN1W3VJ1j', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-09T22:22:00+02:00", "phrase": "Mostly clear", "iconCode": 34, "hasPrecipitation": false, "isDayTime": false, "temperature": {"value": 22.9, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 23.7, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 23.7, "unit": "C", "unitType": 17}, "relativeHumidity": 82, "dewPoint": {"value": 19.7, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 248.0, "localizedDescription": "WSW"}, "speed": {"value": 7.5, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 17.3, "unit": "km/h", "unitType": 7}}, "uvIndex": 0, "uvIndexPhrase": "Low", "visibility": {"value": 24.1, "unit": "km", "unitType": 6}, "obstructionsToVisibi

In [25]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-09 20:30:08+0000
End     : 2025-06-09 20:30:12+0000
Elapsed : 0:00:04  (4.00 seconds)


{'prompt_tokens': 1449, 'completion_tokens': 226, 'total_tokens': 1675, 'prompt_token_details': {'cached_tokens': 0}}

In [26]:
    # Fetch and log all messages
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style.
MessageRole.AGENT: 🌟 Ciao, weather lovers! Here’s your latest scoop from the Eternal City:

Tonight in Rome, the sky is mostly clear—perfect for stargazing while enjoying a late-night gelato. 🍦✨ Temperatures are hanging out at a comfy 23°C (that’s 73°F for my international amici), with just a hint of a cool breeze from the west-southwest at about 7 km/h.🌀

Humidity is cozy at 82%, and there’s absolutely no precipitation in sight. That’s right: leave your umbrellas at home and grab your sunglasses (for tomorrow, at least)! 🕶️

The air pressure is rising, which usually means more stable, beautiful weather is on its way. If you’re out exploring the Colosseum or tossing a coin in the Trevi Fountain, expect great visibility and a gentle Roman night.

That’s the Roman weather wrap for today! Arrivederci—may your evenings be as magical as Rome itself! 🇮🇹🌙


### Post processing

In [27]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_FCLBFiOWAlhV0xeDqTfASuWA, Name: gpt-4o-mini-weather-agent, Model: gpt-4.1, Instructions: You are a weather bot. Use the provided functions to help answer questions.
Agent ID: asst_X14buc7bU593KebYPPXug1Zk, Name: my-agent, Model: gpt-4.1, Instructions: You are a helpful agent
Agent ID: asst_KsSaVO8dGHb4pZgoKfQXJkmB, Name: my-agent, Model: gpt-4.1, Instructions: You are a helpful agent
Agent ID: asst_P9XmgZKtmeioNY3Rka4GZSRz, Name: my-agent, Model: gpt-4.1, Instructions: You are a helpful agent
Agent ID: asst_wiMeqv85Na2yXl0bNSvYZe5T, Name: my-agent, Model: gpt-4.1, Instructions: You are a helpful agent
Agent ID: asst_8asvuTP2VKWlLxPBv8H9lOEd, Name: my-agent, Model: gpt-4.1, Instructions: You are a helpful agent
Agent ID: asst_S1ZolN6lsiQTypNAnbBii2qg, Name: my-agent, Model: gpt-4.1, Instructions: You are a helpful agent
Agent ID: asst_6R9CTPTJrQcpZo06oNldkvn4, Name: my-agent, Model: gpt-4.1, Instructions: You are a helpful agent
Agent ID

In [29]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [30]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")